In [ ]:
from google.colab import drive


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd


In [ ]:
traindata = pd.read_csv('/content/drive/MyDrive/KDDTrain+.txt')

In [ ]:
traindata.head()

,0,tcp,ftp_data,SF,491,0.1,0.2,0.3,0.4,0.5,...,0.17,0.03,0.17.1,0.00.6,0.00.7,0.00.8,0.05,0.00.9,normal,20
0,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.0,0.00,normal,15
1,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.0,0.00,neptune,19
2,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.0,0.01,normal,21
3,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,normal,21
4,0,tcp,private,REJ,0,0,0,0,0,0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.0,1.00,neptune,21


In [ ]:
columns = (['duration'
,'protocol_type'
,'service'
,'flag'
,'src_bytes'
,'dst_bytes'
,'land'
,'wrong_fragment'
,'urgent'
,'hot'
,'num_failed_logins'
,'logged_in'
,'num_compromised'
,'root_shell'
,'su_attempted'
,'num_root'
,'num_file_creations'
,'num_shells'
,'num_access_files'
,'num_outbound_cmds'
,'is_host_login'
,'is_guest_login'
,'count'
,'srv_count'
,'serror_rate'
,'srv_serror_rate'
,'rerror_rate'
,'srv_rerror_rate'
,'same_srv_rate'
,'diff_srv_rate'
,'srv_diff_host_rate'
,'dst_host_count'
,'dst_host_srv_count'
,'dst_host_same_srv_rate'
,'dst_host_diff_srv_rate'
,'dst_host_same_src_port_rate'
,'dst_host_srv_diff_host_rate'
,'dst_host_serror_rate'
,'dst_host_srv_serror_rate'
,'dst_host_rerror_rate'
,'dst_host_srv_rerror_rate'
,'attack'
,'level'])

In [ ]:
traindata.columns = columns


In [ ]:
traindata.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack,level
0,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.0,0.00,normal,15
1,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.0,0.00,neptune,19
2,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.0,0.01,normal,21
3,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,normal,21
4,0,tcp,private,REJ,0,0,0,0,0,0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.0,1.00,neptune,21


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import xgboost as xgb
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, SimpleRNN
from keras.utils import to_categorical
import numpy as np

In [ ]:

traindata_attack = traindata.attack.map(lambda a: 0 if a == 'normal' else 1)

In [ ]:

traindata['attack'] = traindata_attack

In [ ]:
categorical_cols = ['protocol_type', 'service', 'flag', 'land', 'logged_in', 'is_host_login', 'is_guest_login']
for col in categorical_cols:
    le = LabelEncoder()
    traindata[col] = le.fit_transform(traindata[col])

In [ ]:
traindata.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack,level
0,0,2,44,9,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.0,0.00,0,15
1,0,1,49,5,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.0,0.00,1,19
2,0,1,24,9,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.0,0.01,0,21
3,0,1,24,9,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0,21
4,0,1,49,1,0,0,0,0,0,0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.0,1.00,1,21


In [ ]:
X = traindata.drop(columns=['attack'])
y = traindata['attack']

In [ ]:
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=len(y.unique()))
xgb_model.fit(X, y)
feature_importances = xgb_model.feature_importances_

In [ ]:
feature_scores = list(zip(X.columns, feature_importances))


In [ ]:
sorted_feature_scores = sorted(feature_scores, key=lambda x: x[1], reverse=True)

In [ ]:
top_features_with_scores = sorted_feature_scores[:20]
for feature, score in top_features_with_scores:
    print(f"Feature: {feature}, Importance Score: {score}")

Feature: src_bytes, Importance Score: 0.4023140072822571
Feature: protocol_type, Importance Score: 0.13859611749649048
Feature: count, Importance Score: 0.07791511714458466
Feature: level, Importance Score: 0.05602999031543732
Feature: dst_host_srv_rerror_rate, Importance Score: 0.05354972183704376
Feature: dst_host_srv_count, Importance Score: 0.05150774121284485
Feature: logged_in, Importance Score: 0.03358444944024086
Feature: dst_bytes, Importance Score: 0.029826849699020386
Feature: diff_srv_rate, Importance Score: 0.02455620840191841
Feature: dst_host_same_src_port_rate, Importance Score: 0.020948372781276703
Feature: dst_host_same_srv_rate, Importance Score: 0.01703299954533577
Feature: dst_host_srv_serror_rate, Importance Score: 0.012684354558587074
Feature: hot, Importance Score: 0.01203132700175047
Feature: dst_host_serror_rate, Importance Score: 0.01087979320436716
Feature: wrong_fragment, Importance Score: 0.010540622286498547
Feature: serror_rate, Importance Score: 0.00523

In [ ]:
sorted_idx = np.argsort(feature_importances)[-20:]
X_selected = X.iloc[:, sorted_idx]

In [ ]:
print("sorted_idx:")
print(sorted_idx)

sorted_idx:
[36 15 12  3 24  7 37  9 38 33 35 29  5 11 32 40 41 22  1  4]


In [ ]:
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X_selected)

In [ ]:
X_train, X_Val, y_train, y_val = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

In [ ]:
y_train_onehot = to_categorical(y_train)
y_val_onehot = to_categorical(y_val)







In [ ]:
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_Val_reshaped = X_Val.reshape(X_Val.shape[0], X_Val.shape[1], 1)


In [ ]:
autoencoder = Sequential()
autoencoder.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
autoencoder.add(Dense(len(y.unique()), activation='softmax'))
autoencoder.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
autoencoder.fit(X_train, y_train_onehot, epochs=10, batch_size=128,validation_data=(X_Val_reshaped, y_val_onehot))

Epoch 1/10
788/788 [==============================] - 3s 2ms/step - loss: 0.0829 - accuracy: 0.9711 - val_loss: 0.0341 - val_accuracy: 0.9908
Epoch 2/10
788/788 [==============================] - 2s 2ms/step - loss: 0.0252 - accuracy: 0.9927 - val_loss: 0.0228 - val_accuracy: 0.9939
Epoch 3/10
788/788 [==============================] - 1s 2ms/step - loss: 0.0189 - accuracy: 0.9942 - val_loss: 0.0204 - val_accuracy: 0.9942
Epoch 4/10
788/788 [==============================] - 1s 2ms/step - loss: 0.0165 - accuracy: 0.9947 - val_loss: 0.0190 - val_accuracy: 0.9938
Epoch 5/10
788/788 [==============================] - 1s 2ms/step - loss: 0.0150 - accuracy: 0.9952 - val_loss: 0.0173 - val_accuracy: 0.9947
Epoch 6/10
788/788 [==============================] - 2s 3ms/step - loss: 0.0141 - accuracy: 0.9954 - val_loss: 0.0164 - val_accuracy: 0.9946
Epoch 7/10
788/788 [==============================] - 2s 3ms/step - loss: 0.0133 - accuracy: 0.9957 - val_loss: 0.0159 - val_accuracy: 0.9949
Epoch 

In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(64, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
lstm_model.add(Dense(len(y.unique()), activation='softmax'))
lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train_reshaped, y_train_onehot, epochs=10, batch_size=128,validation_data=(X_Val_reshaped, y_val_onehot))

Epoch 1/10
788/788 [==============================] - 28s 33ms/step - loss: 0.1325 - accuracy: 0.9496 - val_loss: 0.0571 - val_accuracy: 0.9779
Epoch 2/10
788/788 [==============================] - 25s 32ms/step - loss: 0.0453 - accuracy: 0.9839 - val_loss: 0.0470 - val_accuracy: 0.9837
Epoch 3/10
788/788 [==============================] - 24s 30ms/step - loss: 0.0319 - accuracy: 0.9892 - val_loss: 0.0287 - val_accuracy: 0.9917
Epoch 4/10
788/788 [==============================] - 25s 31ms/step - loss: 0.0259 - accuracy: 0.9911 - val_loss: 0.0284 - val_accuracy: 0.9908
Epoch 5/10
788/788 [==============================] - 25s 32ms/step - loss: 0.0236 - accuracy: 0.9919 - val_loss: 0.0265 - val_accuracy: 0.9914
Epoch 6/10
788/788 [==============================] - 26s 33ms/step - loss: 0.0210 - accuracy: 0.9929 - val_loss: 0.0251 - val_accuracy: 0.9909
Epoch 7/10
788/788 [==============================] - 23s 29ms/step - loss: 0.0202 - accuracy: 0.9930 - val_loss: 0.0222 - val_accuracy:

In [ ]:
gru_model = Sequential()
gru_model.add(GRU(64, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
gru_model.add(Dense(len(y.unique()), activation='softmax'))
gru_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
gru_model.fit(X_train_reshaped, y_train_onehot, epochs=10, batch_size=128,validation_data=(X_Val_reshaped, y_val_onehot))

Epoch 1/10
788/788 [==============================] - 25s 30ms/step - loss: 0.1289 - accuracy: 0.9454 - val_loss: 0.0654 - val_accuracy: 0.9785
Epoch 2/10
788/788 [==============================] - 20s 26ms/step - loss: 0.0451 - accuracy: 0.9852 - val_loss: 0.0406 - val_accuracy: 0.9868
Epoch 3/10
788/788 [==============================] - 23s 29ms/step - loss: 0.0319 - accuracy: 0.9901 - val_loss: 0.0337 - val_accuracy: 0.9877
Epoch 4/10
788/788 [==============================] - 21s 27ms/step - loss: 0.0249 - accuracy: 0.9920 - val_loss: 0.0265 - val_accuracy: 0.9923
Epoch 5/10
788/788 [==============================] - 24s 30ms/step - loss: 0.0199 - accuracy: 0.9935 - val_loss: 0.0217 - val_accuracy: 0.9935
Epoch 6/10
788/788 [==============================] - 22s 27ms/step - loss: 0.0172 - accuracy: 0.9944 - val_loss: 0.0207 - val_accuracy: 0.9936
Epoch 7/10
788/788 [==============================] - 22s 28ms/step - loss: 0.0147 - accuracy: 0.9948 - val_loss: 0.0173 - val_accuracy:

In [ ]:
rnn_model = Sequential()
rnn_model.add(SimpleRNN(64, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
rnn_model.add(Dense(len(y.unique()), activation='softmax'))
rnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
rnn_model.fit(X_train_reshaped, y_train_onehot, epochs=10, batch_size=128,validation_data=(X_Val_reshaped, y_val_onehot))

Epoch 1/10
788/788 [==============================] - 7s 7ms/step - loss: 0.0461 - accuracy: 0.9838 - val_loss: 0.0225 - val_accuracy: 0.9925
Epoch 2/10
788/788 [==============================] - 7s 9ms/step - loss: 0.0189 - accuracy: 0.9938 - val_loss: 0.0160 - val_accuracy: 0.9946
Epoch 3/10
788/788 [==============================] - 5s 7ms/step - loss: 0.0153 - accuracy: 0.9948 - val_loss: 0.0145 - val_accuracy: 0.9955
Epoch 4/10
788/788 [==============================] - 7s 8ms/step - loss: 0.0131 - accuracy: 0.9954 - val_loss: 0.0137 - val_accuracy: 0.9951
Epoch 5/10
788/788 [==============================] - 6s 7ms/step - loss: 0.0119 - accuracy: 0.9959 - val_loss: 0.0128 - val_accuracy: 0.9955
Epoch 6/10
788/788 [==============================] - 5s 7ms/step - loss: 0.0115 - accuracy: 0.9959 - val_loss: 0.0123 - val_accuracy: 0.9959
Epoch 7/10
788/788 [==============================] - 7s 9ms/step - loss: 0.0107 - accuracy: 0.9962 - val_loss: 0.0122 - val_accuracy: 0.9954
Epoch 

In [ ]:
models = [autoencoder, lstm_model, gru_model, rnn_model]
names = ['Autoencoder', 'LSTM', 'GRU', 'RNN']

for model, name in zip(models, names):
    if name == 'Autoencoder':
        y_pred = model.predict(X_Val)
    else:
        y_pred = model.predict(X_Val_reshaped)
    y_pred_class =  np.argmax(y_pred, axis=1)
    accuracy = accuracy_score(y_val, y_pred_class)
    print(f"Accuracy for {name}: {accuracy}")

788/788 [==============================] - 1s 1ms/step
Accuracy for Autoencoder: 0.9952768406429847
788/788 [==============================] - 3s 4ms/step
Accuracy for LSTM: 0.9940861282000397
788/788 [==============================] - 4s 4ms/step
Accuracy for GRU: 0.9948402460805715
788/788 [==============================] - 2s 2ms/step
Accuracy for RNN: 0.9950783885691605


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
performance_metrics = {}

In [ ]:
y_val_label_encoded = np.argmax(y_val_onehot, axis=1)

In [ ]:
models = [autoencoder, lstm_model, gru_model, rnn_model]
names = ['Autoencoder', 'LSTM', 'GRU', 'RNN']
performance_metrics = {}

y_val_label_encoded = np.argmax(y_val_onehot, axis=1)

for model, name in zip(models, names):
    if name == 'Autoencoder':
        y_pred = model.predict(X_Val)
    else:
        y_pred = model.predict(X_Val_reshaped)

    y_pred_class = np.argmax(y_pred, axis=1)

    # Calculate metrics
    accuracy = accuracy_score(y_val_label_encoded, y_pred_class)
    precision = precision_score(y_val_label_encoded, y_pred_class, average='weighted')
    recall = recall_score(y_val_label_encoded, y_pred_class, average='weighted')
    f1 = f1_score(y_val_label_encoded, y_pred_class, average='weighted')

    # Store metrics in the dictionary
    performance_metrics[name] = {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1 Score': f1}

# Convert the metrics dictionary to a DataFrame for better visualization
performance_df = pd.DataFrame(performance_metrics).transpose()

print(performance_df)

788/788 [==============================] - 2s 3ms/step
             Accuracy  Precision    Recall  F1 Score
Autoencoder  0.995277   0.995288  0.995277  0.995277
LSTM         0.994086   0.994091  0.994086  0.994087
GRU          0.994840   0.994843  0.994840  0.994841
RNN          0.995078   0.995082  0.995078  0.995079


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
models = [autoencoder, lstm_model, gru_model, rnn_model]
names = ['Autoencoder', 'LSTM', 'GRU', 'RNN']


In [ ]:
!pip install seaborn

In [ ]:
testdata =  pd.read_csv('/content/drive/MyDrive/KDDTest+.txt')

In [ ]:
testdata.columns = columns

In [ ]:
testdata.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack,level
0,0,tcp,private,REJ,0,0,0,0,0,0,...,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00,neptune,21
1,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00,normal,21
2,0,icmp,eco_i,SF,20,0,0,0,0,0,...,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00,saint,15
3,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,0.31,0.17,0.03,0.02,0.00,0.0,0.83,0.71,mscan,11
4,0,tcp,http,SF,267,14515,0,0,0,0,...,1.00,0.00,0.01,0.03,0.01,0.0,0.00,0.00,normal,21


In [ ]:
testdata_attack = testdata.attack.map(lambda a: 0 if a == 'normal' else 1)

In [ ]:
testdata['attack'] = testdata_attack

In [ ]:
categorical_cols = ['protocol_type', 'service', 'flag', 'land', 'logged_in', 'is_host_login', 'is_guest_login']
for col in categorical_cols:
    le = LabelEncoder()
    testdata[col] = le.fit_transform(testdata[col])

In [ ]:
X_test = testdata.drop(columns=['attack'])
y_test = testdata['attack']

In [ ]:
X_test_selected = X_test.iloc[:, sorted_idx]

In [ ]:
X_test_normalized = scaler.transform(X_test_selected)

In [ ]:
y_test_onehot = to_categorical(y_test)

In [ ]:
X_test_reshaped = X_test_normalized.reshape(X_test_normalized.shape[0], X_test_normalized.shape[1], 1)


In [ ]:
for model, name in zip(models, names):
    if name == 'Autoencoder':
        y_pred = model.predict(X_test_normalized)
    else:
        y_pred = model.predict(X_test_reshaped)

    y_pred_class = np.argmax(y_pred, axis=1)
    accuracy = accuracy_score(y_test, y_pred_class)
    print(f"Accuracy for {name} on Test Data: {accuracy}")

705/705 [==============================] - 1s 2ms/step
Accuracy for Autoencoder on Test Data: 0.8603114048706916
705/705 [==============================] - 5s 8ms/step
Accuracy for LSTM on Test Data: 0.8453178370225791
705/705 [==============================] - 4s 5ms/step
Accuracy for GRU on Test Data: 0.8333850862795547
705/705 [==============================] - 3s 4ms/step
Accuracy for RNN on Test Data: 0.8313889012110189
